In [1]:
from config import  load_data, evaluate
from dspy_workflow import fin_robot
import dspy

In [2]:
def dspy_evaluate(example, pred, trace=None):
    return evaluate(pred, example.label, example.mode, example.task)

In [3]:
train, val, test = load_data()

In [4]:
train_dspy = [
    dspy.Example(task=x[0]['task'], mode=x[0]['mode'], label=x[1]['label']).with_inputs('task') for x in train
]
test_dspy = [
    dspy.Example(task=x[0]['task'], mode=x[0]['mode'], label=x[1]['label']).with_inputs('task') for x in test
]

In [5]:
train_dspy[:5]

[Example({'task': 'What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nTeollisuuden Voima Oyj , the Finnish utility known as TVO , said it shortlisted Mitsubishi Heavy s EU-APWR model along with reactors from Areva , Toshiba Corp. , GE Hitachi Nuclear Energy and Korea Hydro & Nuclear Power Co. .', 'mode': 'sentiment_analysis', 'label': 'neutral'}) (input_keys={'task'}),
 Example({'task': 'What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nSanofi poaches AstraZeneca scientist as new research head', 'mode': 'sentiment_analysis', 'label': 'neutral'}) (input_keys={'task'}),
 Example({'task': "What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.\nStarbucks says the workers violated safety policies while workers said they'd never heard of the policy before and are alleging retaliat

In [6]:
evaluator = dspy.Evaluate(
    devset=test_dspy,
    metric=dspy_evaluate,
    display_progress=True,
    num_threads=50,
)

In [7]:
evaluator(fin_robot)

Average Metric: 81.72 / 300 (27.2%): 100%|██████████| 300/300 [01:15<00:00,  3.97it/s]

2025/02/09 01:37:20 INFO dspy.evaluate.evaluate: Average Metric: 81.72339144247138 / 300 (27.2%)


27.24

In [7]:
gpt4o = dspy.LM(model='gpt-4o', max_tokens=2048)

models = dict(prompt_model=gpt4o, teacher_settings=dict(lm=gpt4o))
tp = dspy.MIPROv2(metric=dspy_evaluate, auto="medium", num_threads=50, **models)
kwargs = dict(minibatch_size=50, minibatch_full_eval_steps=4, requires_permission_to_run=False)

In [9]:
len(fin_robot.predictors())

12

In [10]:
optimized = tp.compile(fin_robot, trainset=train_dspy, num_trials=64, max_bootstrapped_demos=4, max_labeled_demos=4, **kwargs)

2025/02/09 01:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 2
valset size: 240

2025/02/09 01:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/02/09 01:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/02/09 01:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=2 sets of demonstrations...
2025/02/09 01:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/02/09 01:45:36 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


Bootstrapping set 1/2
Bootstrapping set 2/2
class FinLeader(dspy.Signature):
    """
You are the leader of the following group members:

Member information in format "- <member_name>: <member_role>"
- Value_Factor_Researcher: As a value factor researcher, the individual must possess expertise in financial statement analysis, a strong understanding of valuation metrics, adeptness in Python for quantitative modeling, and the ability to work collaboratively in team settings to integrate the value perspective into broader investment strategies.
- Growth_Factor_Researcher: As a growth factor researcher, the individual must possess expertise in analyzing corporate growth indicators like earnings and revenue expansion, have strong Python skills for data analysis, and collaborate effectively in group settings to evaluate investment growth opportunities.
- Momentum_Factor_Researcher: As a momentum factor researcher, one needs to have the ability to identify and analyze market trends and price p

2025/02/09 01:45:51 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing instructions...

2025/02/09 01:47:26 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/02/09 01:47:26 INFO dspy.teleprompt.mipro_optimizer_v2: 0: You are the leader of the following group members:

Member information in format "- <member_name>: <member_role>"
- Value_Factor_Researcher: As a value factor researcher, the individual must possess expertise in financial statement analysis, a strong understanding of valuation metrics, adeptness in Python for quantitative modeling, and the ability to work collaboratively in team settings to integrate the value perspective into broader investment strategies.
- Growth_Factor_Researcher: As a growth factor researcher, the individual must possess expertise in analyzing corporate growth indicators like earnings and revenue expansion, have strong Python skills for data analysis, and collaborate effectively in group settings to evaluate invest

Average Metric: 78.70 / 240 (32.8%): 100%|██████████| 240/240 [00:26<00:00,  8.94it/s]

2025/02/09 01:47:54 INFO dspy.evaluate.evaluate: Average Metric: 78.69535235418194 / 240 (32.8%)
2025/02/09 01:47:54 INFO dspy.teleprompt.mipro_optimizer_v2: Default program score: 32.79

2025/02/09 01:47:54 INFO dspy.teleprompt.mipro_optimizer_v2: ==> STEP 3: FINDING OPTIMAL PROMPT PARAMETERS <==
2025/02/09 01:47:54 INFO dspy.teleprompt.mipro_optimizer_v2: We will evaluate the program over a series of trials with different combinations of instructions and few-shot examples to find the optimal combination using Bayesian Optimization.

2025/02/09 01:47:54 INFO dspy.teleprompt.mipro_optimizer_v2: == Minibatch Trial 1 / 25 ==


...

2025/02/09 02:02:11 INFO dspy.evaluate.evaluate: Average Metric: 71.19741258878985 / 240 (29.7%)
2025/02/09 02:02:11 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [32.79, 27.56, 28.83, 28.36, 28.0, 29.63, 29.67]
2025/02/09 02:02:11 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 32.79
2025/02/09 02:02:11 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/02/09 02:02:11 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/02/09 02:02:11 INFO dspy.teleprompt.mipro_optimizer_v2: == Minibatch Trial 25 / 25 ==



Average Metric: 19.46 / 50 (38.9%): 100%|██████████| 50/50 [00:03<00:00, 15.46it/s]

2025/02/09 02:02:17 INFO dspy.evaluate.evaluate: Average Metric: 19.461440355492076 / 50 (38.9%)
2025/02/09 02:02:17 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 38.92 on minibatch of size 50 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 0', 'Predictor 1: Instruction 1', 'Predictor 1: Few-Shot Set 1', 'Predictor 2: Instruction 0', 'Predictor 2: Few-Shot Set 0', 'Predictor 3: Instruction 1', 'Predictor 3: Few-Shot Set 0', 'Predictor 4: Instruction 0', 'Predictor 4: Few-Shot Set 0', 'Predictor 5: Instruction 1', 'Predictor 5: Few-Shot Set 0', 'Predictor 6: Instruction 1', 'Predictor 6: Few-Shot Set 1', 'Predictor 7: Instruction 1', 'Predictor 7: Few-Shot Set 0', 'Predictor 8: Instruction 0', 'Predictor 8: Few-Shot Set 1', 'Predictor 9: Instruction 0', 'Predictor 9: Few-Shot Set 0', 'Predictor 10: Instruction 0', 'Predictor 10: Few-Shot Set 0', 'Predictor 11: Instruction 1', 'Predictor 11: Few-Shot Set 0'].
2025/02/09 02:02:17 INFO dspy.teleprompt.mipro_opti


Average Metric: 70.20 / 240 (29.2%): 100%|██████████| 240/240 [00:23<00:00, 10.17it/s]

2025/02/09 02:02:47 INFO dspy.evaluate.evaluate: Average Metric: 70.19741258878985 / 240 (29.2%)
2025/02/09 02:02:47 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [32.79, 27.56, 28.83, 28.36, 28.0, 29.63, 29.67, 29.25]
2025/02/09 02:02:47 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 32.79
2025/02/09 02:02:47 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/02/09 02:02:47 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/02/09 02:02:47 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 32.79!


In [11]:
evaluator(optimized)

Average Metric: 74.26 / 300 (24.8%): 100%|██████████| 300/300 [00:22<00:00, 13.57it/s] 

2025/02/09 02:26:56 INFO dspy.evaluate.evaluate: Average Metric: 74.25712722224446 / 300 (24.8%)


24.75

In [12]:
optimized.save("fin_robot_optimized")